# Purpose

- Choose and clear the data to be used on the RAG pipeline

In [2]:
import pandas as pd

# Define Constant Variables

- SOURCE_PATH -> Refering to the file to process
- TARGET_PATH -> Refering to the saving file

In [3]:
SOURCE_PATH = "../../data/processed/train.csv"
TARGET_PATH = "../../data/ready/train.csv"

# Key functions

In [4]:
def csv_to_dataframe(path_csv: str) -> pd.DataFrame:
    
    df = pd.read_csv(path_csv)
    return df

def save_dataframe_csv(df: pd.DataFrame, path_csv: str):
    df.to_csv(path_csv, index=False)
    print(f"[OK] CSV salvo em {path_csv}")

In [8]:
def extract_key_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a new DF with the key features:
      - REF_DATE
      - TARGET
      - VAR2
      - IDADE
      - VAR4
      - VAR5
      - VAR8
    """
    cols = ["REF_DATE", "TARGET", "VAR2", "IDADE", "VAR4", "VAR5", "VAR8"]
    
    # Quick validation
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise KeyError(f"As colunas a seguir não foram encontradas no DataFrame: {missing}")
    return df[cols].copy()

In [5]:
df = csv_to_dataframe(SOURCE_PATH)

## Recap of Data

### Simple Visualization

In [6]:
df.head()

,REF_DATE,TARGET,VAR2,IDADE,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,...,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,ID
0,2017-06-01 00:00:00+00:00,0,M,34.137,NaN,RO,-8.808779,-63.878470,D,E,...,2680.289259,D,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,181755
1,2017-08-18 00:00:00+00:00,0,M,40.447,NaN,PB,-7.146537,-34.926080,E,E,...,1777.725469,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,287633
2,2017-06-30 00:00:00+00:00,0,F,33.515,NaN,RS,-27.900178,-53.314035,NaN,E,...,1695.494979,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,88015
3,2017-08-05 00:00:00+00:00,1,F,25.797,NaN,BA,-12.948874,-38.451863,E,E,...,1399.037809,E,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE#@#NOME INEXISTENTE#@#CEP INE...,2.6.1,122576
4,2017-07-29 00:00:00+00:00,0,F,54.074,NaN,RS,-30.051810,-51.213277,B,E,...,7868.793296,C,NaN,NaN,NaN,NaN,102,EMAIL INEXISTENTE,2.6.1,1272


In [19]:
df.shape

(120750, 151)

In [20]:
df.columns

Index(['REF_DATE', 'TARGET', 'VAR2', 'IDADE', 'VAR4', 'VAR5', 'VAR6', 'VAR7',
       'VAR8', 'VAR9',
       ...
       'VAR141', 'VAR142', 'VAR143', 'VAR144', 'VAR145', 'VAR146', 'VAR147',
       'VAR148', 'VAR149', 'ID'],
      dtype='object', length=151)

In [21]:
df.dtypes

REF_DATE     object
TARGET        int64
VAR2         object
IDADE       float64
VAR4         object
             ...   
VAR146      float64
VAR147        int64
VAR148       object
VAR149       object
ID            int64
Length: 151, dtype: object

## Selection of the Key metrics

- REF_DATE  (Data de referência do registro)
- TARGET    (Alvo binário de inadimplência (1: Mau Pagador, i.e. atrado > 60 dias em 2 meses))
- VAR2      (Sexo)
- IDADE     (Idade do Individuo)
- VAR4      (Flag de óbito)
- VAR5      (Unidade Federativa (UF) brasileira)
- VAR8      (Classe social estimada)

In [9]:
filtered_df = extract_key_features(df)

In [10]:
filtered_df.head()

,REF_DATE,TARGET,VAR2,IDADE,VAR4,VAR5,VAR8
0,2017-06-01 00:00:00+00:00,0,M,34.137,NaN,RO,D
1,2017-08-18 00:00:00+00:00,0,M,40.447,NaN,PB,E
2,2017-06-30 00:00:00+00:00,0,F,33.515,NaN,RS,NaN
3,2017-08-05 00:00:00+00:00,1,F,25.797,NaN,BA,E
4,2017-07-29 00:00:00+00:00,0,F,54.074,NaN,RS,B


The data seems fine, however, can be improvided... For example, we can remove the time line from REF_DATE, change Idade to an INT, change VAR4 to a Dead or Alive...

In [11]:
filtered_df.shape

(120750, 7)

In [13]:
filtered_df.dtypes

REF_DATE     object
TARGET        int64
VAR2         object
IDADE       float64
VAR4         object
VAR5         object
VAR8         object
dtype: object

## Correcting the Ref metric

In [18]:
filtered_df["REF_DATE"] = pd.to_datetime(filtered_df["REF_DATE"], utc=True).dt.strftime("%d-%m-%Y")

In [33]:
filtered_df.head()

,REF_DATE,Inadiplencia,VAR2,IDADE,VAR4,VAR5,VAR8
0,01-06-2017,adimplente,M,34.137,NaN,RO,D
1,18-08-2017,adimplente,M,40.447,NaN,PB,E
2,30-06-2017,adimplente,F,33.515,NaN,RS,NaN
3,05-08-2017,inadiplente,F,25.797,NaN,BA,E
4,29-07-2017,adimplente,F,54.074,NaN,RS,B


In [35]:
filtered_df["REF_DATE"][filtered_df["REF_DATE"].isnull()]

Series([], Name: REF_DATE, dtype: object)

## Correcting the Target

In [20]:
filtered_df["TARGET"].unique()

array([0, 1])

No problem here... So, to help the LLM, let's change the binary to Inadimplente or Adimplente

In [21]:
filtered_df.rename(columns={"TARGET": "Inadiplencia"}, inplace=True)

filtered_df["Inadiplencia"] = filtered_df["Inadiplencia"].map({
    0: "adimplente",
    1: "inadiplente"
})

In [22]:
filtered_df["Inadiplencia"].head()

0     adimplente
1     adimplente
2     adimplente
3    inadiplente
4     adimplente
Name: Inadiplencia, dtype: object

In [36]:
filtered_df["Inadiplencia"][filtered_df["Inadiplencia"].isnull()]

Series([], Name: Inadiplencia, dtype: object)

## Correcting VAR2

In [23]:
filtered_df["VAR2"].unique()

array(['M', 'F', nan], dtype=object)

Because it has nan symbol, i will check the amount, and validate if can be change to "Non-specified"

In [27]:
filtered_df["VAR2"] = filtered_df["VAR2"].fillna("Non-specified")

In [28]:
counts_var2 = filtered_df["VAR2"].value_counts()
print(counts_var2)

VAR2
F                60131
M                46000
Non-specified    14619
Name: count, dtype: int64


A lot of missing data... So, Non-specified suits well...

In [37]:
filtered_df.rename(columns={"VAR2": "Sexo"}, inplace=True)

In [38]:
filtered_df["Sexo"][filtered_df["Sexo"].isnull()]

Series([], Name: Sexo, dtype: object)

## Correcting Idade

In [39]:
filtered_df["IDADE"] = filtered_df["IDADE"].fillna(filtered_df["IDADE"].median().round().astype(int))

In [41]:
filtered_df["IDADE"].sample(5).head()

118071    49.233
91175     72.540
64548     66.578
39395     51.901
66583     64.488
Name: IDADE, dtype: float64

In [42]:
filtered_df["IDADE"][filtered_df["IDADE"].isnull()]

Series([], Name: IDADE, dtype: float64)

In [43]:
filtered_df["IDADE"] = filtered_df["IDADE"].round().astype("Int16")

In [45]:
filtered_df["IDADE"].sample(5).head()

49836     19
96256     71
109712    57
105174    49
39236     40
Name: IDADE, dtype: Int16

## Correcting VAR4

In [ ]:
filtered_df.rename(columns={"VAR4": "Obito"}, inplace=True)

In [51]:
filtered_df["Obito"][filtered_df["Obito"].isnull()].sum()

0

If everything is good, everyone is Alive :D

In [ ]:
filtered_df["Obito"] = filtered_df["Obito"].fillna("Vivo")

In [56]:
filtered_df["Obito"].head()

0    Vivo
1    Vivo
2    Vivo
3    Vivo
4    Vivo
Name: Obito, dtype: object

## Correcting VAR5

In [57]:
filtered_df.rename(columns={"VAR5": "UF"}, inplace=True)

In [58]:
filtered_df["UF"].unique()

array(['RO', 'PB', 'RS', 'BA', 'SP', 'MG', 'RJ', 'MS', 'PR', 'SC', 'RN',
       nan, 'CE', 'AC', 'PE', 'TO', 'AL', 'MT', 'ES', 'MA', 'AM', 'PI',
       'DF', 'GO', 'PA', 'SE', 'RR', 'AP'], dtype=object)

We have nan, so lets check it

In [61]:
filtered_df["UF"].value_counts()

UF
SP    19079
BA    10306
PA    10159
RS     8410
CE     8262
MG     7757
PE     7056
RJ     4617
AM     4145
RN     3800
PR     3668
PB     3085
AL     2800
ES     2765
MS     2515
GO     2369
AC     2304
MA     2083
MT     2015
SC     2013
PI     1858
RO     1731
AP     1292
SE     1175
DF      790
TO      767
RR      573
Name: count, dtype: int64

In [62]:
filtered_df["UF"].isnull().sum()

np.int64(3356)

3000 it's a lot, so, let's create a new category to fit it.

In [65]:
filtered_df["UF"].fillna("Non-specified", inplace=True)

In [66]:
filtered_df["UF"].isnull().sum()

np.int64(0)

## Correcting VAR8

In [67]:
filtered_df.rename(columns={"VAR8": "Classe Social"}, inplace=True)

In [68]:
filtered_df["Classe Social"].isnull().sum()

np.int64(53110)

That's a huge number, let me compare to the complete dataset..

In [70]:
int(filtered_df.shape[0]) - int(filtered_df["Classe Social"].isnull().sum())

67640

We don't have nearly half of the dataset, i will placed a N/S to avoid getting wrong data, even if i consider statistics

In [72]:
filtered_df["Classe Social"].fillna("Non-specified", inplace=True)

In [74]:
filtered_df["Classe Social"].isnull().sum()

np.int64(0)

In [78]:
filtered_df["Classe Social"].value_counts()

Classe Social
E                54928
Non-specified    53110
D                 9130
C                 2972
B                  449
A                  161
Name: count, dtype: int64

# Saving the "Ready" version

What was done...

Solved most of the issues with the dataset, and prepare the data to be sent to a DB...

The columns that changed, follows

- REF_DATE  (Data de referência do registro)
- TARGET    (Alvo binário de inadimplência (1: Mau Pagador, i.e. atrado > 60 dias em 2 meses))
- VAR2      (Sexo)
- IDADE     (Idade do Individuo)
- VAR4      (Flag de óbito)
- VAR5      (Unidade Federativa (UF) brasileira)
- VAR8      (Classe social estimada)

In [80]:
save_dataframe_csv(filtered_df, TARGET_PATH)

[OK] CSV salvo em ../../data/ready/train.csv
